# Edit Your Expectation Suite
Use this notebook to recreate and modify your expectation suite:

**Expectation Suite Name**: `input_data_snowflake_check`


In [36]:
import datetime

import pandas as pd

import great_expectations as gx
import great_expectations.jupyter_ux
from great_expectations.core.expectation_configuration import ExpectationConfiguration
from great_expectations.data_context.types.resource_identifiers import ExpectationSuiteIdentifier
from great_expectations.exceptions import DataContextError
from great_expectations.core.batch import BatchRequest
from great_expectations.checkpoint import SimpleCheckpoint

context = gx.get_context()


# Feel free to change the name of your suite here. Renaming this will not remove the other one.
expectation_suite_name = "input_data_snowflake_check"
try:
    suite = context.get_expectation_suite(expectation_suite_name=expectation_suite_name)
    print(f'Loaded ExpectationSuite "{suite.expectation_suite_name}" containing {len(suite.expectations)} expectations.')
except DataContextError:
    suite = context.add_expectation_suite(expectation_suite_name=expectation_suite_name)
    print(f'Created ExpectationSuite "{suite.expectation_suite_name}".')

2023-07-26T20:56:53+0000 - INFO - FileDataContext loading fluent config
2023-07-26T20:56:53+0000 - INFO - Loading 'datasources' ->
[{'name': 'default_pandas_datasource', 'type': 'pandas'}]
2023-07-26T20:56:53+0000 - INFO - Of 1 entries, no 'datasources' could be loaded
Created ExpectationSuite "input_data_snowflake_check".


In [37]:
batch_request = {'datasource_name': 'my_snowflake_datasource', 'data_connector_name': 'default_inferred_data_connector_name', 'data_asset_name': 'raw.inventory', 'limit': 1000}

validator = context.get_validator(
    batch_request=BatchRequest(**batch_request),
    expectation_suite_name=expectation_suite_name
)
column_names = [f'"{column_name}"' for column_name in validator.columns()]
print(f"Columns: {', '.join(column_names)}.")
validator.head(n_rows=5, fetch_all=False)

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

Columns: "cal_dt", "store_key", "prod_key", "inventory_on_hand_qty", "inventory_on_order_qty", "out_of_stock_flg", "waste_qty", "promotion_flg", "next_delivery_dt".


Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,cal_dt,store_key,prod_key,inventory_on_hand_qty,inventory_on_order_qty,out_of_stock_flg,waste_qty,promotion_flg,next_delivery_dt
0,2020-01-01,248,539839,33.28,28.16,1,0.0,True,2009-01-14
1,2020-01-01,248,1064589,7.56,8.19,0,1.0,False,2009-01-06
2,2020-01-01,1054,539839,38.40,57.60,0,1.0,True,2009-01-10
3,2020-01-01,1054,1064589,13.68,7.92,1,1.0,True,2009-01-16
4,2020-01-01,1103,539839,8.64,48.96,1,1.0,False,2009-01-15


## Create & Edit Expectations


You are adding Expectation configurations to the suite. Since you selected manual mode, there is no sample batch of data and no validation happens during this process. See our documentation for more info and examples: **[How to create a new Expectation Suite without a sample batch](https://docs.greatexpectations.io/docs/guides/expectations/how_to_create_and_edit_expectations_based_on_domain_knowledge_without_inspecting_data_directly)**.

Note that if you do use interactive mode you may specify a sample batch of data to use when creating your Expectation Suite. You can then use a `validator` to get immediate feedback on your Expectations against your specified sample batch.


You can see all the available expectations in the **[expectation gallery](https://greatexpectations.io/expectations)**.

### Table Expectation(s)

No table level expectations are in this suite. Feel free to add some here.


### Column Expectation(s)

No column level expectations are in this suite. Feel free to add some here.


In [38]:
expectations=suite.expectations

In [39]:
expectations.append(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_not_be_null",
        kwargs={
            "column":"prod_key"
        }
    )
)

In [40]:
expectations.append(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_in_set",
        kwargs={
            "column": "out_of_stock_flg",
            "value_set": [0,1],
        },
        # This is how you can optionally add a comment about this expectation. It will be rendered in Data Docs.
    meta={
        "notes": {
            "format": "markdown",
            "content": "This is ensuring that this flag is properly set to 1 or 2",
        }
    },
    )
)

In [41]:
expectations.append(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column": "store_key",
            "min_value": 100,
            "max_value": 9999,
        },
    meta={
        "notes": {
            "format": "markdown",
            "content": "Store number should be 4 digits long",
        }
    },
    )
)

In [42]:
expectations.append(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column": "inventory_on_hand_qty",
            "min_value": 0,
            "max_value": 999999,
        },
    meta={
        "notes": {
            "format": "markdown",
            "content": "Ensure inventory on hand is > 0",
        }
    },
    )
)

## Review & Save Your Expectations

Let's save the expectation suite as a JSON file in the `great_expectations/expectations` directory of your project.

Let's now rebuild your Data Docs, which helps you communicate about your data with both machines and humans.

In [43]:
print(context.get_expectation_suite(expectation_suite_name=expectation_suite_name))
context.add_or_update_expectation_suite(expectation_suite=suite)

suite_identifier = ExpectationSuiteIdentifier(expectation_suite_name=expectation_suite_name)
context.build_data_docs(resource_identifiers=[suite_identifier])
context.open_data_docs(resource_identifier=suite_identifier)

{
  "expectation_suite_name": "input_data_snowflake_check",
  "ge_cloud_id": null,
  "expectations": [],
  "data_asset_type": null,
  "meta": {
    "great_expectations_version": "0.17.6"
  }
}


In [44]:
#suite.expectations=expectations

# save suite of expectations
#context.save_expectation_suite(suite, "input_data_snowflake_check")